In [1]:
# ============================================================================
# ⚙️ 기본 라이브러리 및 시스템 유틸리티
# ============================================================================
import os
import gc
import warnings
import time
import json
import joblib
import traceback
from dotenv import load_dotenv
from collections import Counter
from numba import jit # JIT 컴파일러
from datetime import datetime, timedelta # 날짜/시간 처리

# ============================================================================
# 📊 데이터 처리 및 분석
# ============================================================================
import numpy as np
import pandas as pd
import pandas_ta as ta # 기술적 분석 라이브러리

# ============================================================================
# 📈 머신러닝 (Scikit-learn)
# ============================================================================

# 📌 전처리 및 선택
from sklearn.feature_selection import (
    SelectKBest, RFE,
    mutual_info_classif, mutual_info_regression
)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler,MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight # 클래스 불균형 처리

# 📌 모델 (기본)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron

# 📌 모델 (앙상블)
from sklearn.ensemble import (
    AdaBoostClassifier, AdaBoostRegressor,
    BaggingClassifier, BaggingRegressor,
    ExtraTreesClassifier, ExtraTreesRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    HistGradientBoostingClassifier, # 히스토그램 기반 GB
    RandomForestClassifier, RandomForestRegressor,
    StackingClassifier, StackingRegressor,
    VotingClassifier, VotingRegressor
)

# 📌 평가 지표
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, # 분류
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, # 회귀
    classification_report
)

# ============================================================================
# 🚀 부스팅 계열 모델
# ============================================================================
import optuna # 하이퍼파라미터 최적화

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm.callback import early_stopping
from xgboost import XGBClassifier, XGBRegressor

# ============================================================================
# 🧠 딥러닝 (TensorFlow/Keras)
# ============================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    # 기본
    Input, Dense, Flatten, Dropout, Activation,
    # RNN/시퀀스
    LSTM, GRU, SimpleRNN, Bidirectional, TimeDistributed, RepeatVector,
    # CNN
    Conv1D, MaxPooling1D, AveragePooling1D,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    # 정규화
    BatchNormalization, LayerNormalization,
    # 유틸리티
    Concatenate, Add, Multiply, Lambda, Reshape, Permute
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2, l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ============================================================================
# ⏱️ 시계열 분석 (Statsmodels)
# ============================================================================
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR

# ============================================================================
# ⚡ PyTorch (선택적)
# ============================================================================
try:
    import torch
    import torch.nn as nn
except ImportError:
    # PyTorch가 설치되지 않은 경우 무시
    pass



2025-12-01 14:15:18.488157: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 14:15:18.488203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 14:15:18.489521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-01 14:15:18.496376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-01 14:15:19.273953: W tensorflow/comp

In [2]:
# ==========================================================================
# 1. 유틸리티 함수 (지표 추가용)
# ==========================================================================

def add_indicator_to_df(df_ta, indicator):
    """pandas_ta 지표 결과를 DataFrame에 안전하게 추가"""
    if indicator is None:
        return

    if isinstance(indicator, pd.DataFrame) and not indicator.empty:
        for col in indicator.columns:
            df_ta[col] = indicator[col]
    elif isinstance(indicator, pd.Series) and not indicator.empty:
        colname = indicator.name if indicator.name else 'Unnamed'
        df_ta[colname] = indicator

def safe_add(df_ta, func, *args, **kwargs):
    """지표 생성 시 오류 방지를 위한 래퍼 함수"""
    try:
        result = func(*args, **kwargs)
        add_indicator_to_df(df_ta, result)
        return True
    except Exception as e:
        return False

# ==========================================================================
# 2. 기술적 지표 생성 (Sniper Optimization)
# ==========================================================================

def calculate_technical_indicators(df):
    """
    [1억 현물 투자용 최적화 지표]
    - 중복 지표(TEMA, DEMA 등) 제거
    - Swing High/Low Breakout(돌파 매매) 지표 추가
    """
    df = df.sort_values('date').reset_index(drop=True)
    df_ta = df.copy()

    close = df['ETH_Close']
    high = df.get('ETH_High', close)
    low = df.get('ETH_Low', close)
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    open_ = df.get('ETH_Open', close)

    # 1. [변동성] ATR 및 볼린저 밴드
    df_ta['ATR_14'] = ta.atr(high, low, close, length=14)
    bb = ta.bbands(close, length=20, std=2)
    if bb is not None:
        df_ta['BB_WIDTH'] = bb.iloc[:, 2]  # Bandwidth

    # 2. [추세] 신뢰도 높은 이평선만 유지
    df_ta['SMA_20'] = ta.sma(close, length=20)
    df_ta['SMA_50'] = ta.sma(close, length=50)
    df_ta['SMA_100'] = ta.sma(close, length=100)
    df_ta['SMA_200'] = ta.sma(close, length=200)
    
    df_ta['EMA_12'] = ta.ema(close, length=12)
    
    # 추세 점수 (정배열 여부)
    df_ta['TREND_SCORE'] = (
        (close > df_ta['SMA_20']).astype(int)
        + (df_ta['SMA_20'] > df_ta['SMA_50']).astype(int)
        + (df_ta['SMA_50'] > df_ta['SMA_100']).astype(int)
    )

    # 3. [모멘텀] RSI, MACD
    df_ta['RSI_14'] = ta.rsi(close, length=14)
    safe_add(df_ta, ta.macd, close, fast=12, slow=26, signal=9)

    # 4. [거래량] OBV, MFI, 상대 거래량
    df_ta['OBV'] = ta.obv(close, volume)
    df_ta['MFI_14'] = ta.mfi(high, low, close, volume, length=14)
    df_ta['VOLUME_RATIO'] = volume / (volume.rolling(20).mean() + 1e-8)

    # 5. [패턴] 윗꼬리/아랫꼬리
    body_high = np.maximum(close, open_)
    body_low = np.minimum(close, open_)
    total_range = (high - low + 1e-9)

    df_ta['UPPER_SHADOW'] = (high - body_high) / total_range
    df_ta['LOWER_SHADOW'] = (body_low - low) / total_range
    df_ta['BODY_RANGE'] = (body_high - body_low) / total_range
    
    # 6. [핵심] Swing Breakout (저항/지지 돌파)
    for window in [5, 20, 60]:
        swing_high = high.rolling(window).max().shift(1)
        swing_low = low.rolling(window).min().shift(1)
        
        # 위치 (1.0 돌파 시 매수 신호)
        df_ta[f'PRICE_VS_HIGH_{window}d'] = close / (swing_high + 1e-9)
        df_ta[f'PRICE_VS_LOW_{window}d'] = close / (swing_low + 1e-9)
        
        # 돌파 강도 (ATR 대비)
        df_ta[f'BREAKOUT_STR_{window}d'] = (close - swing_high) / (df_ta['ATR_14'] + 1e-9)

    return df_ta

# ==========================================================================
# 3. 파생 변수 및 전처리 함수 
# ==========================================================================

def add_enhanced_cross_crypto_features(df):
    df_enhanced = df.copy()
    df_enhanced['eth_return'] = df['ETH_Close'].pct_change().fillna(0)
    
    if 'BTC_Close' in df.columns:
        df_enhanced['btc_return'] = df['BTC_Close'].pct_change().fillna(0)
        # 상관관계
        for window in [7, 30]:
            df_enhanced[f'eth_btc_corr_{window}d'] = (
                df_enhanced['eth_return'].rolling(window).corr(df_enhanced['btc_return'])
            ).fillna(0)
        # 스프레드
        df_enhanced['eth_btc_spread'] = df_enhanced['eth_return'] - df_enhanced['btc_return']
        
    return df_enhanced

def add_price_lag_features_first(df):
    """가격 패턴 학습용 Lag"""
    df_new = df.copy()
    close = df['ETH_Close']
    # 핵심적인 1, 2, 3, 5, 10일 전만 확인
    for lag in [1, 2, 3, 5, 10]:
        df_new[f'return_lag{lag}'] = close.pct_change(periods=lag).shift(1)
    return df_new

def add_interaction_features(df):
    df_interact = df.copy()
    # 거래량 실린 지표 확인
    if 'RSI_14' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['RSI_Volume_Strength'] = df['RSI_14'] * df['VOLUME_RATIO']
    return df_interact

def add_volatility_regime_features(df):
    df_regime = df.copy()
    # 변동성 체제 확인
    if 'ATR_14' in df.columns:
        atr_ma = df['ATR_14'].rolling(20).mean()
        df_regime['high_volatility_regime'] = (df['ATR_14'] > atr_ma).astype(int)
    return df_regime

def add_percentile_features(df):
    df_pct = df.copy()
    # 현재 가격 위치 (랭크)
    if 'ETH_Close' in df.columns:
        df_pct['price_rank_250d'] = df['ETH_Close'].rolling(250).rank(pct=True)
    return df_pct

def preprocess_non_stationary_features(df):
    """외부 데이터(TVL 등) 정규화"""
    df_proc = df.copy()
    
    prefixes_to_transform = [
        'eth_', 'aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_',
        'l2_', 'sp500_', 'gold_', 'dxy_', 'vix_', 'usdt_'
    ]
    
    exclude_prefixes = ['fg_', 'funding_']
    exclude_keywords = [
        '_pct_', '_ratio', '_lag', '_volatility', '_corr', '_beta', '_spread',
        'eth_return', 'btc_return', 'eth_log_return',
        'RSI', 'MFI', 'CCI', 'ADX', 'BOP', 'AROON', 'PRICE_VS', 'BREAKOUT','funding','fg_','kimchi_'
    ]
    
    cols_to_transform = []
    for col in df_proc.columns:
        col_lower = col.lower()
        if col.startswith(tuple(prefixes_to_transform)):
            if not col.startswith(tuple(exclude_prefixes)):
                if not any(k.lower() in col_lower for k in exclude_keywords):
                    cols_to_transform.append(col)
                    
    cols_to_drop = []
    for col in cols_to_transform:
        series = df_proc[col].fillna(method='ffill').replace(0, 1e-9)
        # 1일 변화율
        df_proc[f'{col}_pct_1d'] = series.pct_change(1)
        # 30일 이동평균 대비 비율
        ma_30 = series.rolling(window=30, min_periods=10).mean()
        df_proc[f'{col}_ma30_ratio'] = series / (ma_30 + 1e-9)
        cols_to_drop.append(col)

    df_proc = df_proc.drop(columns=cols_to_drop, errors='ignore')
    df_proc = df_proc.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df_proc

# ==========================================================================
# 4. 결측치 및 초기 데이터 처리 (Burn-in)
# ==========================================================================

def handle_missing_values_paper_based(df_clean, train_start_date, is_train=True, train_stats=None):
    """초기 200일 데이터 제거 (안정성 확보)"""
    burn_in_period = 200
    if len(df_clean) > burn_in_period:
        df_clean = df_clean.sort_values('date').reset_index(drop=True)
        df_clean = df_clean.iloc[burn_in_period:].reset_index(drop=True)

    if isinstance(train_start_date, str):
        train_start_date = pd.to_datetime(train_start_date)
    df_clean = df_clean[df_clean['date'] >= train_start_date].reset_index(drop=True)
    
    target_cols = ['next_log_return', 'next_direction', 'next_close','next_open', 
                   'take_profit_price', 'stop_loss_price', 'exit_reason']
    feature_cols = [col for col in df_clean.columns if col not in target_cols + ['date']]
    
    df_clean[feature_cols] = df_clean[feature_cols].fillna(method='ffill').fillna(0)
    df_clean[feature_cols] = df_clean[feature_cols].replace([np.inf, -np.inf], 0)
    
    if is_train:
        return df_clean, {}
    else:
        return df_clean


# ==========================================================================
# 5. 데이터 누수 방지 및 정제
# ==========================================================================

def remove_raw_prices_and_transform(df, target_type, method):
    df_transformed = df.copy()
    
    if 'eth_log_return' not in df_transformed.columns:
        df_transformed['eth_log_return'] = np.log(df['ETH_Close'] / df['ETH_Close'].shift(1))
    
    remove_patterns = ['_Close', '_Open', '_High', '_Low', '_Volume']
    keep_keywords = [
        '_lag', '_position', '_ratio', '_range', '_change', '_corr', '_volatility', '_obv',
        'PRICE_VS', 'BREAKOUT', 'UPPER_SHADOW', 'LOWER_SHADOW', 'BB_WIDTH'
    ]
    
    cols_to_remove = [
        col for col in df_transformed.columns
        if any(p in col for p in remove_patterns)
        and not any(d in col.lower() for d in [k.lower() for k in keep_keywords])
    ]
    df_transformed.drop(cols_to_remove, axis=1, inplace=True)

    return_cols = [col for col in df_transformed.columns if 'return' in col.lower() and 'next' not in col]
    if return_cols:
        df_transformed[return_cols] = df_transformed[return_cols].fillna(0)

    return df_transformed


In [3]:
def add_kimchi_premium_features(df):
    df_k = df.copy()

    required_cols = ['ETH_Close', 'ETH_binance_Close', 'USDKRW_Close']
    if not all(c in df_k.columns for c in required_cols):
        # 환율이 없으면 BTC 크로스로 대체하는 로직을 따로 짜도 됨
        return df_k

    # 해외 기준 ETH 가격(KRW 환산)
    global_eth_krw = df_k['ETH_binance_Close'] * df_k['USDKRW_Close']

    # 김치 프리미엄 (비율, %) 
    df_k['kimchi_eth_premium'] = (
        (df_k['ETH_Close'] - global_eth_krw) / (global_eth_krw + 1e-9)
    )
    df_k['kimchi_eth_premium_pct'] = df_k['kimchi_eth_premium'] * 100

    # 김프 레짐/과열 구간 (예: |김프| > 5% 이상)
    df_k['kimchi_eth_extreme'] = (df_k['kimchi_eth_premium'].abs() > 0.05).astype(int)

    # 김프 추세 (5/20일 평균)
    df_k['kimchi_eth_premium_ma5'] = df_k['kimchi_eth_premium'].rolling(5).mean()
    df_k['kimchi_eth_premium_ma20'] = df_k['kimchi_eth_premium'].rolling(20).mean()

    return df_k


In [4]:
from numba import jit
import numpy as np
import pandas as pd
import pandas_ta as ta

from numba import jit
import numpy as np

@jit(nopython=True)
def compute_targets_with_hourly_numba(
    daily_entry_ts,   # ★ t일 09:00 기준 타임스탬프
    daily_atr,
    daily_open,
    hourly_dates_ts,
    hourly_high,
    hourly_low,
    lookahead_days,
    profit_mult,
    stop_mult
):
    n = len(daily_entry_ts)
    targets = np.full(n, -1, dtype=np.int32)
    entry_prices = np.full(n, np.nan)
    tp_prices = np.full(n, np.nan)
    sl_prices = np.full(n, np.nan)

    one_day_ms = 86400000
    lookahead_ms = lookahead_days * one_day_ms

    h_start = 0

    # 마지막 날은 t+1일 일봉 없음
    for i in range(n - 1):
        atr = daily_atr[i]
        if np.isnan(atr) or atr <= 0:
            continue

        # t일 정보 → t+1일 09:00 진입
        entry_start_ts = daily_entry_ts[i] + one_day_ms      # t+1 09:00
        entry_end_ts   = entry_start_ts + lookahead_ms       # t+1 09:00 ~ t+1+L 09:00

        # t+1일부터의 첫 1시간봉 인덱스
        first_entry_idx = -1
        for h in range(h_start, len(hourly_dates_ts)):
            if hourly_dates_ts[h] >= entry_start_ts:
                first_entry_idx = h
                h_start = h
                break

        if first_entry_idx == -1:
            continue

        entry_price = daily_open[i + 1]   # t+1일 일봉 시가
        tp = entry_price + (atr * profit_mult)
        sl = entry_price - (atr * stop_mult)

        entry_prices[i] = entry_price
        tp_prices[i] = tp
        sl_prices[i] = sl

        result = 0
        for h in range(first_entry_idx, len(hourly_dates_ts)):
            ts = hourly_dates_ts[h]
            if ts >= entry_end_ts:
                break

            if hourly_low[h] <= sl:
                result = 0
                break
            if hourly_high[h] >= tp:
                result = 1
                break

        targets[i] = result

    return targets, entry_prices, tp_prices, sl_prices



def create_targets(df, hourly_df, lookahead=5, profit_mult=1.5, stop_mult=1.0, **kwargs):
    df_target = df.copy()
    hourly_df = hourly_df.copy()
    
    df_target['date'] = pd.to_datetime(df_target['date'])
    hourly_df['datetime'] = pd.to_datetime(hourly_df['datetime'])
    
    hourly_df.columns = hourly_df.columns.str.lower()
    
    required_hourly_cols = ['datetime', 'open', 'high', 'low']
    for col in required_hourly_cols:
        if col not in hourly_df.columns:
            raise ValueError(f"1시간봉 데이터에 '{col}' 컬럼이 없습니다. 현재 컬럼: {hourly_df.columns.tolist()}")
    
    required_daily_cols = ['date', 'ETH_High', 'ETH_Low', 'ETH_Close', 'ETH_Open']
    for col in required_daily_cols:
        if col not in df_target.columns:
            raise ValueError(f"일봉 데이터에 '{col}' 컬럼이 없습니다. 현재 컬럼: {df_target.columns.tolist()}")
    
    if 'ATR_14' not in df_target.columns:
        df_target['ATR_14'] = ta.atr(
            df_target['ETH_High'], df_target['ETH_Low'], df_target['ETH_Close'], length=14
        )
    
    df_target = df_target.sort_values('date').reset_index(drop=True)
    hourly_df = hourly_df.sort_values('datetime').reset_index(drop=True)
    
    daily_min = df_target['date'].min()
    daily_max = df_target['date'].max()
    hourly_min = hourly_df['datetime'].min()
    hourly_max = hourly_df['datetime'].max()
    
    print(f"일봉 기간: {daily_min.date()} ~ {daily_max.date()}")
    print(f"1시간봉 기간 (KST 변환 후): {hourly_min} ~ {hourly_max}")
    
    if hourly_min > daily_min + pd.Timedelta(days=1):
        print(f"[경고] 1시간봉 시작일({hourly_min})이 일봉 시작일({daily_min}) 이후입니다. 초반 타겟 손실 발생.")
    
    daily_dates_ts = df_target['date'].astype(np.int64).values // 10**6
    daily_atr = df_target['ATR_14'].fillna(method='ffill').fillna(0).to_numpy()
    daily_open = df_target['ETH_Open'].astype(np.float64).to_numpy()  # ★ 추가
    
    hourly_dates_ts = hourly_df['datetime'].astype(np.int64).values // 10**6
    hourly_high = hourly_df['high'].astype(np.float64).to_numpy()
    hourly_low = hourly_df['low'].astype(np.float64).to_numpy()
    
    targets, entry_prices, tp_prices, sl_prices = compute_targets_with_hourly_numba(
        daily_dates_ts, daily_atr, daily_open,
        hourly_dates_ts, hourly_high, hourly_low,
        lookahead, profit_mult, stop_mult
    )
    
    df_target['next_direction'] = targets
    df_target['real_entry_price'] = entry_prices
    df_target['take_profit_price'] = tp_prices
    df_target['stop_loss_price'] = sl_prices
    
    df_target['next_close'] = df_target['ETH_Close'].shift(-1)
    df_target['next_open'] = df_target['ETH_Open'].shift(-1)
    df_target['next_log_return'] = np.log(df_target['next_close'] / (df_target['next_open'] + 1e-9))
    
    if lookahead > 0:
        df_target = df_target.iloc[:-lookahead]
    
    valid_before = len(df_target)
    df_target = df_target[df_target['next_direction'] != -1].reset_index(drop=True)
    valid_after = len(df_target)
    
    removed = valid_before - valid_after
    win = (df_target['next_direction'] == 1).sum()
    lose = (df_target['next_direction'] == 0).sum()
    total = win + lose
    
    print(f"유효 샘플: {valid_after}/{valid_before} (제거: {removed})")
    print(f"Win: {win} | Lose: {lose} | Win Rate: {win/total:.2%}" if total > 0 else "타겟 생성 실패")
    
    return df_target


In [5]:
import numpy as np
import pandas as pd

import numpy as np

def remove_correlated_features(
    X,
    threshold=0.9,
    verbose=True,
    core_keep=None,   # 이더리움 핵심 피처 화이트리스트
):
    """
    피처들끼리의 상관계수(|r|)가 threshold 이상이면
    한 쪽을 제거해서 다중공선성/중복을 줄인다.
    
    단, core_keep에 포함된 '핵심 피처'는 상관이 높더라도 제거 대상에서 보호한다.
    """
    # 1) 기본 코어 피처 세트 (ETH 전략용)
    if core_keep is None:
        core_keep = {
            # 변동성 / 리스크
            'ATR_14',
            'BB_WIDTH',

            # 가격 모멘텀 / 리턴
            'eth_return',
            'btc_return',
            'eth_log_return',

            # 오실레이터 / 모멘텀
            'RSI_14',
            'MACD_12_26_9',

            # 추세 / 위치
            'SMA_200',
            'SMA_100',
            'PRICE_VS_HIGH_60d',
            'PRICE_VS_LOW_60d',
            'PRICE_VS_HIGH_20d',
            'PRICE_VS_LOW_5d',
            'BREAKOUT_STR_20d',
            'BREAKOUT_STR_60d',
            'price_rank_250d',

            # 거래량/수급
            'OBV',
            'VOLUME_RATIO',

            # ETH-BTC 관계
            'eth_btc_corr_7d',
            'eth_btc_corr_30d',
            'eth_btc_spread',

            # 온체인 / TVL / 스테이블
            'usdt_totalBridgedToUSD_ma30_ratio',
            'usdt_totalUnreleased_ma30_ratio',
            'usdt_totalMintedUSD_ma30_ratio',
            'curve_curve-dex_eth_tvl_ma30_ratio',
            'makerdao_makerdao_eth_tvl_ma30_ratio',
            'l2_optimism_tvl_ma30_ratio',
            'l2_base_tvl_ma30_ratio',

            # 매크로 / 크로스에셋
            'gold_GOLD_ma30_ratio',
            'dxy_DXY_ma30_ratio',
            'sp500_SP500_ma30_ratio',
            'vix_VIX_ma30_ratio',
        }

    # 실제 존재하는 컬럼만 대상으로 필터 적용
    core_keep = {c for c in core_keep if c in X.columns}

    # 2) 상관계수 행렬 (절대값)
    corr_matrix = X.corr().abs()

    # 상삼각 행렬만 사용 (자기 자신/중복 제외)
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )

    to_drop = []
    for col in upper.columns:
        if col in core_keep:
            # 코어 피처는 어떤 경우에도 제거하지 않음
            continue
        # col이 다른 어떤 피처와라도 threshold 이상 상관이면 드랍
        if any(upper[col] >= threshold):
            to_drop.append(col)

    if verbose:
        print(f"[Multicollinearity] threshold={threshold}")
        print(f" -> 보호 피처 수(존재하는 것만): {len(core_keep)}")
        if len(core_keep) > 0:
            print(f" -> 예시(core_keep): {list(core_keep)[:10]}")
        print(f" -> 제거 대상 피처 수: {len(to_drop)}")
        if len(to_drop) > 0:
            print(f" -> 제거 피처 예시: {to_drop[:10]}")

    X_reduced = X.drop(columns=to_drop)
    return X_reduced, to_drop



from collections import Counter
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import RFE
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

def select_features_verified(X_train, y_train, task='class', top_n=20, verbose=True):
    if task == 'class':
        mi_scores = mutual_info_classif(X_train, y_train, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_train, y_train, random_state=42, n_neighbors=3)
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    estimator = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1) if task == 'class' else LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    rfe = RFE(estimator=estimator, n_features_to_select=top_n, step=0.1, verbose=0)
    rfe.fit(X_train, y_train)
    rfe_features = X_train.columns[rfe.support_].tolist()

    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1) if task == 'class' else RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    rf_idx = np.argsort(rf_model.feature_importances_)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]
    
    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0: break
    
    return selected_features, {}

def select_features_multi_target(
    X_train, y_train, target_type='direction', top_n=20,
    corr_threshold=0.9
):
    atr_col_name = 'ATR_14'

    # 1) 먼저 다중공선성 필터
    X_base, dropped_corr = remove_correlated_features(
        X_train, threshold=corr_threshold, verbose=True
    )
    
    if target_type == 'direction':
        selected, stats = select_features_verified(
            X_base, y_train['next_direction'], task='class', top_n=top_n
        )
        
        # ATR_14는 강제로 포함
        if atr_col_name not in selected and atr_col_name in X_base.columns:
            if len(selected) > 0:
                selected.pop()
            selected.insert(0, atr_col_name)
    
    print(f"\n[Feature Selection] Top {len(selected)} Features Selected:")
    print(f" -> {', '.join(selected)}")
    return selected, stats

In [6]:

def process_single_split(split_data, target_type='direction', top_n=20, fold_idx=None, trend_params=None): 
    
    train_df = split_data['train'] 
    val_df = split_data['val'] 
    test_df = split_data['test'] 
    fold_type = split_data.get('fold_type', 'unknown')

    print(f"\n{'='*80}")
    print(f" Processing Fold {fold_idx} ({fold_type})")
    print(f"{'='*80}")
    print(f" Train Period: {train_df['date'].min().date()} ~ {train_df['date'].max().date()} (N={len(train_df)})")
    print(f" Val   Period: {val_df['date'].min().date()} ~ {val_df['date'].max().date()} (N={len(val_df)})")
    print(f" Test  Period: {test_df['date'].min().date()} ~ {test_df['date'].max().date()} (N={len(test_df)})")

    train_processed, missing_stats = handle_missing_values_paper_based(train_df, train_df['date'].min(), is_train=True)
    val_processed = handle_missing_values_paper_based(val_df, val_df['date'].min(), is_train=False, train_stats=missing_stats)
    test_processed = handle_missing_values_paper_based(test_df, test_df['date'].min(), is_train=False, train_stats=missing_stats)

    target_cols = [
        'next_direction', 'next_log_return', 'next_close', 'next_open', 
        'take_profit_price', 'stop_loss_price', 
        'ATR_14', 'real_entry_price' 
    ]

    train_processed = train_processed.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_processed.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_processed.dropna(subset=target_cols).reset_index(drop=True)

    # [수정 4] ATR_14는 피처로도 써야 하므로 exclude_cols에서 제외
    exclude_cols = [col for col in target_cols if col != 'ATR_14'] + ['date']
    feature_cols = [col for col in train_processed.columns if col not in exclude_cols]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]

    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]

    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    balance = y_train['next_direction'].value_counts(normalize=True).to_dict()
    print(f"[Class Balance] Train Set: {balance}")

    selected_features, selection_stats = select_features_multi_target(
        X_train, y_train, target_type=target_type, top_n=top_n
    )

    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]


    scaler = MinMaxScaler(feature_range=(-1, 1))

    X_train_scaled = scaler.fit_transform(X_train_sel)
    X_val_scaled = scaler.transform(X_val_sel)
    X_test_scaled = scaler.transform(X_test_sel)


    X_train_final = pd.DataFrame(X_train_scaled, columns=selected_features)
    X_val_final = pd.DataFrame(X_val_scaled, columns=selected_features)
    X_test_final = pd.DataFrame(X_test_scaled, columns=selected_features)

    # ML용 Raw 데이터 (필요시 사용, 여기선 선택사항)
    X_train_raw = X_train_sel 

    if trend_params is None:
        trend_params = {}

    result = {
        'train': {
            # [수정 3] 모든 키가 MinMax Scaled 데이터를 가리키게 함
            'X_robust': X_train_final,   
            'X_standard': X_train_final, 
            'X_minmax': X_train_final,
            'X_raw': X_train_raw,
            'y': y_train.reset_index(drop=True), 
            'dates': train_processed['date'].reset_index(drop=True) 
        },
        'val': {
            'X_robust': X_val_final,
            'X_standard': X_val_final,
            'X_minmax': X_val_final,
            'X_raw': X_val_sel,
            'y': y_val.reset_index(drop=True), 
            'dates': val_processed['date'].reset_index(drop=True)
        },
        'test': {
            'X_robust': X_test_final,
            'X_standard': X_test_final,
            'X_minmax': X_test_final,
            'X_raw': X_test_sel,
            'y': y_test.reset_index(drop=True),
            'dates': test_processed['date'].reset_index(drop=True)
        },
        'scaler': scaler, 
        'stats': {
            'robust_scaler': scaler, 
            'standard_scaler': scaler,
            'minmax_scaler': scaler,
            'selected_features': selected_features,
            'selection_stats': selection_stats,
            'missing_stats': missing_stats,  
            'target_type': target_type,
            'target_cols': target_cols,
            'fold_type': fold_type,
            'fold_idx': fold_idx,
            'trend_window': trend_params.get('trend_window', 120),
            'trend_analysis_points': trend_params.get('trend_analysis_points', 5)
        }
    }

    return result


def split_walk_forward_method(df, train_start_date, final_test_start='2025-01-01', 
                              initial_train_size=800, val_size=90, test_size=90, 
                              step=90, gap_size=7):
    
    df_period = df[df['date'] >= train_start_date].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    total_days = len(df_period)
    
    print(f"\n{'='*80}")
    print(f"Reverse Rolling Walk-Forward + Final Holdout")
    print(f"{'='*80}")
    print(f"Total period: {df_period['date'].min().date()} ~ {df_period['date'].max().date()} ({total_days} days)")
    print(f"Rolling train: {initial_train_size}d | Val: {val_size}d | Test: {test_size}d | Gap: {gap_size}d")
    print(f"Final holdout test: {final_test_start.date()} ~ {df_period['date'].max().date()}")
    print(f"{'='*80}\n")
    
    # ========== 1. Walk-Forward Folds (전체 데이터로 rolling) ==========
    folds = []
    current_test_end = total_days
    
    while True:
        test_end_idx = current_test_end
        test_start_idx = test_end_idx - test_size
        
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0:
            break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_type': 'walk_forward_rolling_reverse'
        })
        
        current_test_end = test_start_idx - gap_size
    
    folds.reverse()
    
    # 폴드 인덱스 할당
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        print(f"Fold {fold['fold_idx']} (walk_forward_rolling)")
        print(f"  Train: {len(fold['train']):4d}d  {fold['train']['date'].min().date()} ~ {fold['train']['date'].max().date()}")
        print(f"  Val:   {len(fold['val']):4d}d  {fold['val']['date'].min().date()} ~ {fold['val']['date'].max().date()}")
        print(f"  Test:  {len(fold['test']):4d}d  {fold['test']['date'].min().date()} ~ {fold['test']['date'].max().date()}\n")
    
    # ========== 2. Final Holdout (2025-01-01 이후 전체를 별도 테스트) ==========
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    
    if len(final_test_df) > 0:
        # Final holdout의 train/val은 final_test_start 직전 데이터 사용
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        
        if len(pre_final_df) >= (initial_train_size + val_size + gap_size):
            # 충분한 데이터가 있으면 정상적으로 train/val 구성
            final_val_end_idx = len(pre_final_df)
            final_val_start_idx = final_val_end_idx - val_size
            final_train_end_idx = final_val_start_idx - gap_size
            final_train_start_idx = final_train_end_idx - initial_train_size
            
            if final_train_start_idx < 0:
                final_train_start_idx = 0
            
            final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
            final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
        else:
            # 데이터 부족 시 가능한 만큼 사용
            split_point = int(len(pre_final_df) * 0.8)
            final_train_data = pre_final_df.iloc[:split_point].copy()
            final_val_data = pre_final_df.iloc[split_point:].copy()
        
        print(f"\n{'='*80}")
        print(f"Fold {len(folds) + 1} (final_holdout - 2025 전체 별도 평가)")
        print(f"{'='*80}")
        print(f"  Train: {len(final_train_data):4d}d  {final_train_data['date'].min().date()} ~ {final_train_data['date'].max().date()}")
        print(f"  Val:   {len(final_val_data):4d}d  {final_val_data['date'].min().date()} ~ {final_val_data['date'].max().date()}")
        print(f"  Test:  {len(final_test_df):4d}d  {final_test_df['date'].min().date()} ~ {final_test_df['date'].max().date()}")
        print(f"{'='*80}\n")
        
        folds.append({
            'train': final_train_data,
            'val': final_val_data,
            'test': final_test_df,
            'fold_idx': len(folds) + 1,
            'fold_type': 'final_holdout'
        })
    
    print(f"{'='*80}")
    print(f"Created {len(folds)} folds total")
    print(f"  - {len(folds)-1} walk-forward folds")
    print(f"  - 1 final holdout fold (2025 전체)")
    print(f"{'='*80}\n")
    
    return folds


def build_complete_pipeline_corrected(df_raw,df_hour, train_start_date, **kwargs): 
    
    
    print(f"\n Pipeline Started... (Train Start: {train_start_date})")

    df = df_raw.copy()
    lookahead = kwargs.get('lookahead_candles', kwargs.get('lookahead', 5))
    profit_mult = kwargs.get('atr_multiplier_profit', 2.0)
    stop_mult = kwargs.get('atr_multiplier_stop', 1.0)

    df = add_price_lag_features_first(df)
    df = calculate_technical_indicators(df)
    df = add_enhanced_cross_crypto_features(df)
    df = add_volatility_regime_features(df)
    df = add_interaction_features(df)
    df = add_percentile_features(df)
    df = preprocess_non_stationary_features(df)

    df = create_targets(df, df_hour,
        lookahead=lookahead, 
        profit_mult=profit_mult, 
        stop_mult=stop_mult
    )

    df = remove_raw_prices_and_transform(df, 'direction', 'tvt')
    print(df.shape)
    splits = split_walk_forward_method(
        df, 
        train_start_date=train_start_date,
        final_test_start=kwargs.get('final_test_start', '2025-01-01'),
        initial_train_size=800,
        val_size=90,
        test_size=90,
        step=90,
        gap_size=lookahead
    )
    print(f" Data Split Completed. Total {len(splits)} folds generated.")

    result = []
    for fold in splits:
        res = process_single_split(
            fold, 
            top_n=kwargs.get('top_n', 20), 
            fold_idx=fold['fold_idx']
        )
        result.append(res)

    return result

In [7]:
import optuna
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, log_loss
import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import LSTM, BatchNormalization, Dropout, Dense
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

class DirectionModels:
    
    @staticmethod
    def get_class_weights(y_train):
        classes = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        return dict(zip(classes, weights))

    # ---------------- LightGBM ----------------
    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 100),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_child_samples': trial.suggest_int('min_child_samples', 30, 150),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0, log=True),
                'objective': 'binary',
                'metric': 'auc',              # AUC로 모니터링
                'class_weight': 'balanced',
                'verbosity': -1,
                'n_jobs': -1,
                'random_state': 42
            }
            
            model = LGBMClassifier(**params)
            callbacks = [lgb.early_stopping(stopping_rounds=30, verbose=False)]
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
            
            val_prob = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, val_prob)

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=25)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 
            'objective': 'binary',
            'metric': 'auc',
            'class_weight': 'balanced', 
            'verbosity': -1, 
            'n_jobs': -1, 
            'random_state': 42
        })
        
        final_model = LGBMClassifier(**best_params)
        callbacks = [lgb.early_stopping(stopping_rounds=50, verbose=False)]
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
        
        # 필요하면 내부 디버깅용으로만 AUC/LogLoss 정도 출력 (선택사항)
        val_prob = final_model.predict_proba(X_val)[:, 1]
        print(f"[LightGBM] Val AUC: {roc_auc_score(y_val, val_prob):.4f} | LogLoss: {log_loss(y_val, val_prob):.4f}")
        
        return final_model

    # ---------------- XGBoost ----------------
    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
                'gamma': trial.suggest_float('gamma', 0.5, 10.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0, log=True),
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'tree_method': 'hist',
                'early_stopping_rounds': 30,
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            
            val_prob = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, val_prob)

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=25)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 
            'objective': 'binary:logistic',
            'eval_metric': 'auc', 
            'tree_method': 'hist', 
            'early_stopping_rounds': 50,
            'random_state': 42, 
            'n_jobs': -1
        })
        
        final_model = XGBClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        
        val_prob = final_model.predict_proba(X_val)[:, 1]
        print(f"[XGBoost] Val AUC: {roc_auc_score(y_val, val_prob):.4f} | LogLoss: {log_loss(y_val, val_prob):.4f}")
        
        return final_model

    # ---------------- CatBoost ----------------
    @staticmethod
    def catboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'iterations': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'depth': trial.suggest_int('depth', 4, 9),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 15),
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
                'loss_function': 'Logloss',
                'eval_metric': 'AUC',
                'auto_class_weights': 'Balanced',
                'logging_level': 'Silent',
                'random_seed': 42,
                'od_type': 'Iter',
                'od_wait': 30,
                'allow_writing_files': False
            }
            
            model = CatBoostClassifier(**params)
            model.fit(X_train, y_train, eval_set=(X_val, y_val))
            
            val_prob = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, val_prob)

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_params = study.best_params
        best_params.update({
            'iterations': 1000, 
            'loss_function': 'Logloss', 
            'eval_metric': 'AUC',
            'auto_class_weights': 'Balanced', 
            'logging_level': 'Silent',
            'random_seed': 42, 
            'od_type': 'Iter', 
            'od_wait': 50, 
            'allow_writing_files': False
        })
        
        final_model = CatBoostClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=(X_val, y_val))
        
        val_prob = final_model.predict_proba(X_val)[:, 1]
        print(f"[CatBoost] Val AUC: {roc_auc_score(y_val, val_prob):.4f} | LogLoss: {log_loss(y_val, val_prob):.4f}")
        
        return final_model

    # ---------------- LSTM ----------------
    @staticmethod
    def lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        class_weights = DirectionModels.get_class_weights(y_train)

        def objective(trial):
            tf.keras.backend.clear_session()
            
            units = trial.suggest_int('units', 32, 128, step=16)
            dropout_rate = trial.suggest_float('dropout', 0.3, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 1e-4, 1e-1, log=True)
            lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
            
            model = Sequential([
                Input(shape=input_shape),
                LSTM(units, return_sequences=False, 
                     kernel_regularizer=l1_l2(l2=l2_reg),
                     recurrent_regularizer=l1_l2(l2=l2_reg)),
                BatchNormalization(),
                Dropout(dropout_rate),
                Dense(units // 2, activation='relu', kernel_regularizer=l1_l2(l2=l2_reg)),
                Dropout(dropout_rate),
                Dense(1, activation='sigmoid')
            ])
            
            model.compile(
                optimizer=Adam(learning_rate=lr),
                loss='binary_crossentropy',
                metrics=[tf.keras.metrics.AUC(name='auc')]
            )
            
            es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            
            model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=20,
                batch_size=64,
                class_weight=class_weights,
                callbacks=[es],
                verbose=0
            )
            
            val_prob = model.predict(X_val, verbose=0).flatten()
            return roc_auc_score(y_val, val_prob)

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best = study.best_params
        tf.keras.backend.clear_session()
        
        final_model = Sequential([
            Input(shape=input_shape),
            LSTM(best['units'], return_sequences=False,
                 kernel_regularizer=l1_l2(l2=best['l2_reg']),
                 recurrent_regularizer=l1_l2(l2=best['l2_reg'])),
            BatchNormalization(),
            Dropout(best['dropout']),
            Dense(best['units'] // 2, activation='relu', kernel_regularizer=l1_l2(l2=best['l2_reg'])),
            Dropout(best['dropout']),
            Dense(1, activation='sigmoid')
        ])
        
        final_model.compile(
            optimizer=Adam(learning_rate=best['lr']),
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.AUC(name='auc')]
        )
        
        final_es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
        final_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        
        final_model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=64,
            class_weight=class_weights,
            callbacks=[final_es, final_lr],
            verbose=0
        )
        
        val_prob = final_model.predict(X_val, verbose=0).flatten()
        print(f"[LSTM] Val AUC: {roc_auc_score(y_val, val_prob):.4f} | LogLoss: {log_loss(y_val, val_prob):.4f}")
        
        return final_model

    
    
ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    #{'index': 2, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 3, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 4, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True}
]

DL_MODELS_CLASSIFICATION = [
    #{'index': 9, 'name': 'LSTM', 'func': DirectionModels.lstm, 'needs_val': True},
]

In [8]:
class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.predictions = {}
        self.models = {} if save_models else None
        self.save_models = save_models
        self.best_thresholds = {}

    def optimize_threshold(self, y_true, buy_prob, min_trades=5, reward_risk_ratio=2.0):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        y_true = np.asarray(y_true, dtype=int)
        buy_prob = np.asarray(buy_prob, dtype=float)

        def objective(trial):
            th = trial.suggest_float('threshold', 0.2, 0.9)
            preds = (buy_prob >= th).astype(int)
            n_trades = np.sum(preds == 1)

            if n_trades < min_trades:
                return -99999.0

            wins = np.sum((preds == 1) & (y_true == 1))
            losses = n_trades - wins
            total_return = (wins * reward_risk_ratio) - (losses * 1.0)
            return total_return

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=30)

        best_th = study.best_params['threshold']
        best_val_return = study.best_value

        # ★ 모든 trial이 페널티만 반환한 경우 → 기본 threshold 사용
        if best_val_return <= -99998.0:
            # 예: 기본값 0.5, 혹은 buy_prob의 상위 60% 분위수 등으로 정의
            default_th = 0.5
            best_th = default_th
            best_val_return = -99999.0  # 의미 없음 표시
        return best_th, best_val_return

    def _calculate_metrics(self, y_true, buy_prob, dataset_name, threshold, reward_risk_ratio):
        metrics = {}
        
        preds = (buy_prob >= threshold).astype(int)
        n_trades = np.sum(preds == 1)
        
        if n_trades > 0:
            wins = np.sum((preds == 1) & (y_true == 1))
            losses = n_trades - wins
            
            win_rate = wins / n_trades
            total_return_r = (wins * reward_risk_ratio) - (losses * 1.0)
            profit_factor = (wins * reward_risk_ratio) / (losses * 1.0) if losses > 0 else 99.0
            expectancy = total_return_r / n_trades
        else:
            win_rate = 0.0
            expectancy = 0.0
            total_return_r = 0.0
            profit_factor = 0.0
            wins = 0
            losses = 0
            
        # [중요] 여기서 컬럼명이 '{dataset_name}_Expectancy'로 생성됩니다.
        metrics[f'{dataset_name}_Trades'] = n_trades
        metrics[f'{dataset_name}_Wins'] = wins
        metrics[f'{dataset_name}_Losses'] = losses
        metrics[f'{dataset_name}_WinRate'] = win_rate
        metrics[f'{dataset_name}_Expectancy'] = expectancy
        metrics[f'{dataset_name}_TotalReturn_R'] = total_return_r
        metrics[f'{dataset_name}_ProfitFactor'] = profit_factor
            
        return metrics

    def _extract_target(self, y_data):
        if isinstance(y_data, pd.DataFrame):
            if 'next_direction' in y_data.columns:
                return y_data['next_direction'].values.astype(int)
            return y_data.iloc[:, 0].values.astype(int)
        elif isinstance(y_data, pd.Series):
            return y_data.values.astype(int)
        elif isinstance(y_data, np.ndarray):
            if y_data.ndim > 1:
                return y_data[:, 0].astype(int)
            return y_data.astype(int)
        return np.array(y_data).astype(int)

    def evaluate_model(self, model, X_train, y_train, X_val, y_val, X_test, y_test, 
                       test_dates, test_prices, test_atr, model_name,
                       is_deep_learning=False, profit_mult=1.5, stop_mult=1.0):
        
        rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5

        if is_deep_learning:
            train_prob = model.predict(X_train, verbose=0).flatten()
            val_prob = model.predict(X_val, verbose=0).flatten()
            test_prob = model.predict(X_test, verbose=0).flatten()
        else:
            train_prob = model.predict_proba(X_train)[:, 1]
            val_prob = model.predict_proba(X_val)[:, 1]
            test_prob = model.predict_proba(X_test)[:, 1]

        y_train_arr = self._extract_target(y_train)
        y_val_arr = self._extract_target(y_val)
        y_test_arr = self._extract_target(y_test)

        best_th, best_val_return = self.optimize_threshold(y_val_arr, val_prob, min_trades=30, reward_risk_ratio=rr_ratio)
        self.best_thresholds[model_name] = best_th
        
        train_metrics = self._calculate_metrics(y_train_arr, train_prob, 'Train', best_th, rr_ratio)
        val_metrics = self._calculate_metrics(y_val_arr, val_prob, 'Val', best_th, rr_ratio)
        test_metrics = self._calculate_metrics(y_test_arr, test_prob, 'Test', best_th, rr_ratio)
        
        result = {'Model': model_name, 'Threshold': best_th, 'RR_Ratio': rr_ratio}
        result.update(train_metrics)
        result.update(val_metrics)
        result.update(test_metrics)
        
        self.results.append(result)
        
        test_preds = (test_prob >= best_th).astype(int)
        
        if hasattr(test_prices, 'values'): test_prices = test_prices.values
        if hasattr(test_atr, 'values'): test_atr = test_atr.values
        test_prices = np.array(test_prices).reshape(-1)
        test_atr = np.array(test_atr).reshape(-1)

        tp_prices = test_prices + (test_atr * profit_mult)
        sl_prices = test_prices - (test_atr * stop_mult)
        
        self.predictions[model_name] = pd.DataFrame({
            'date': test_dates,
            'actual': y_test_arr,
            'pred': test_preds,
            'prob': test_prob,
            'entry_price': test_prices,
            'tp_price': tp_prices,
            'sl_price': sl_prices,
            'atr': test_atr
        })
        
        if self.save_models and self.models is not None:
            self.models[model_name] = model
        print(
            f"[{model_name}] Th={best_th:.3f} | "
            f"Train W:{train_metrics['Train_WinRate']:.2%} "
            f"Val W:{val_metrics['Val_WinRate']:.2%} "
            f"Test W:{test_metrics['Test_WinRate']:.2%} "
            f"| Test Exp:{test_metrics['Test_Expectancy']:.3f}R "
            f"PF:{test_metrics['Test_ProfitFactor']:.2f} "
            f"Trades:{test_metrics['Test_Trades']}"
        )

        return result

    def get_summary_dataframe(self):
        return pd.DataFrame(self.results)
    
    def get_predictions_dict(self):
        return self.predictions
    
    def get_models_dict(self):
        return self.models or {}

# -----------------------------------------------------------------------------
# 2. ModelTrainer Class (모델 학습 래퍼)
# -----------------------------------------------------------------------------
class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    def create_sequences(self, X, y, lookback):
        Xs, ys = [], []
        for i in range(lookback, len(X)):
            Xs.append(X[i-lookback:i])
            ys.append(y[i])
        return np.array(Xs), np.array(ys)
    
    def _prepare_target(self, y_data):
        if isinstance(y_data, pd.DataFrame):
            if 'next_direction' in y_data.columns:
                return y_data['next_direction'].values.astype(int)
            return y_data.iloc[:, 0].values.astype(int)
        elif isinstance(y_data, pd.Series):
            return y_data.values.astype(int)
        return np.array(y_data).astype(int)

    def train_all_models(self, X_train, y_train, X_val, y_val, X_test, y_test,
                         test_dates, test_prices, test_atr,
                         profit_mult, stop_mult,
                         ml_models=None, dl_models=None):
        
        y_train_arr = self._prepare_target(y_train)
        y_val_arr = self._prepare_target(y_val)
        y_test_arr = self._prepare_target(y_test)
        
        # ML Models
        if ml_models:
            for config in ml_models:
                try:
                    if config['needs_val']:
                        model = config['func'](X_train, y_train_arr, X_val, y_val_arr)
                    else:
                        model = config['func'](X_train, y_train_arr)
                        
                    self.evaluator.evaluate_model(
                        model, X_train, y_train_arr, X_val, y_val_arr, X_test, y_test_arr,
                        test_dates, test_prices, test_atr,
                        config['name'],
                        is_deep_learning=False,
                        profit_mult=profit_mult,
                        stop_mult=stop_mult
                    )
                    del model
                    gc.collect()
                except Exception as e:
                    print(f"[Error] {config['name']}: {str(e)}")
                    traceback.print_exc()

        # DL Models
        if dl_models:
            X_train_seq, y_train_seq = self.create_sequences(X_train, y_train_arr, self.lookback)
            X_val_seq, y_val_seq = self.create_sequences(X_val, y_val_arr, self.lookback)
            X_test_seq, y_test_seq = self.create_sequences(X_test, y_test_arr, self.lookback)
            
            test_dates_seq = test_dates[self.lookback:]
            test_prices_seq = test_prices[self.lookback:]
            test_atr_seq = test_atr[self.lookback:]
            
            input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
            
            for config in dl_models:
                try:
                    tf.keras.backend.clear_session()
                    model = config['func'](X_train_seq, y_train_seq, X_val_seq, y_val_seq, input_shape)
                    
                    self.evaluator.evaluate_model(
                        model, X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq,
                        test_dates_seq, test_prices_seq, test_atr_seq,
                        config['name'],
                        is_deep_learning=True,
                        profit_mult=profit_mult,
                        stop_mult=stop_mult
                    )
                    del model
                    tf.keras.backend.clear_session()
                    gc.collect()
                except Exception as e:
                    print(f"[Error] {config['name']}: {str(e)}")
                    traceback.print_exc()


def save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data):
    base_dir = f"model_results/{datetime.now().strftime('%Y-%m-%d')}/{trial_name}/fold_{fold_idx}_{fold_type}"
    os.makedirs(base_dir, exist_ok=True)
    
    summary = evaluator.get_summary_dataframe()
    summary.to_csv(f"{base_dir}/fold_summary.csv", index=False)
    
    # 1) 예측 저장
    for name, df in evaluator.get_predictions_dict().items():
        df.to_csv(f"{base_dir}/predictions_{name}.csv", index=False)
        
    # 2) 모델 저장
    for name, model in evaluator.get_models_dict().items():
        try:
            if isinstance(model, tf.keras.Model):
                model.save(f"{base_dir}/model_{name}.h5")
            else:
                joblib.dump(model, f"{base_dir}/model_{name}.pkl")
        except Exception as e:
            print(f"[Warning] Failed to save {name}: {e}")
    
    # 3) 스케일러 저장
    if 'scaler' in fold_data:
        joblib.dump(fold_data['scaler'], f"{base_dir}/scaler.pkl")
    
    # 4) 모델별 하이퍼파라미터 추출
    model_params = {}
    for name, model in evaluator.get_models_dict().items():
        try:
            # sklearn / XGB / LGBM / CatBoost 계열은 보통 get_params 지원[web:250][web:255][web:256][web:268]
            if hasattr(model, "get_params"):
                params = model.get_params()
            else:
                params = {}
        except Exception:
            params = {}
        model_params[name] = params
    
    # 5) 메타데이터 저장
    meta_data = {
        'selected_features': fold_data['stats']['selected_features'],
        'target_cols': fold_data['stats']['target_cols'],
        'fold_idx': fold_idx,
        'fold_type': fold_type,
        'trial_name': trial_name,
        'best_thresholds': evaluator.best_thresholds,
        'model_params': model_params,   # ★ 여기 추가
    }
    
    with open(f"{base_dir}/metadata.json", 'w') as f:
        json.dump(meta_data, f, indent=4)
            
    return summary



In [9]:
def run_optuna_optimization(df_merged, df_hour, n_trials=30):
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d")
    RESULT_DIR = f"model_results/{TIMESTAMP}"
    os.makedirs(RESULT_DIR, exist_ok=True)
    
    LOG_PATH = f"{RESULT_DIR}/optuna_log.csv"
    
    existing_history = pd.DataFrame()
    if os.path.exists(LOG_PATH):
        try:
            existing_history = pd.read_csv(LOG_PATH)
            print(f"\n[Resume] Loaded {len(existing_history)} existing trials from {LOG_PATH}")
        except Exception:
            print("\n[Warning] Log file exists but could not be read.")

    if not os.path.exists(LOG_PATH):
        with open(LOG_PATH, "w") as f:
            f.write("trial,lookahead,profit_mult,stop_mult,score\n")

    def objective(trial):
        nonlocal existing_history 
        
        lookahead = trial.suggest_int('lookahead', 5, 12)
        p_mult   = trial.suggest_float('profit_mult', 1.5, 2.5, step=0.2)
        s_mult   = trial.suggest_float('stop_mult',   1.0, 2.0, step=0.2)

        if not existing_history.empty:
            mask = (
                (existing_history['lookahead'] == lookahead) &
                (np.isclose(existing_history['profit_mult'], p_mult, atol=1e-5)) &
                (np.isclose(existing_history['stop_mult'], s_mult, atol=1e-5))
            )
            if mask.any():
                past_score = existing_history.loc[mask, 'score'].values[0]
                print(f"\n[Skip] Found existing result for L={lookahead}, P={p_mult:.1f}, S={s_mult:.1f}. Score: {past_score:.4f}")
                return past_score

        unique_id = datetime.now().strftime('%H%M%S')
        trial_name = f"trial_{trial.number}_{unique_id}_L{lookahead}_P{p_mult:.1f}_S{s_mult:.1f}"
        
        print(f"\n{'='*80}")
        print(f" Starting NEW {trial_name}")
        
        try:
            pipeline_result = build_complete_pipeline_corrected(
                df_raw=df_merged,
                df_hour=df_hour,
                train_start_date='2020-01-01',
                final_test_start='2025-01-01',
                lookahead_candles=lookahead,
                atr_multiplier_profit=p_mult,
                atr_multiplier_stop=s_mult,
                top_n=20
            )
            
            model_scores = {
                'CatBoost': [],
                'LightGBM': [],
                'XGBoost': []
            }
            
            for i, fold_data in enumerate(pipeline_result):
                fold_idx = fold_data['stats']['fold_idx']
                fold_type = fold_data['stats']['fold_type']
                
                print(f"\n   >> Running Fold {fold_idx}/{len(pipeline_result)} ({fold_type})")
                
                evaluator = ModelEvaluator(save_models=True)
                trainer = ModelTrainer(evaluator, lookback=30)
                
                test_prices = fold_data['test']['y']['real_entry_price'].values
                test_atr = fold_data['test']['y']['ATR_14'].values
                
                trainer.train_all_models(
                    fold_data['train']['X_robust'],
                    fold_data['train']['y'],
                    fold_data['val']['X_robust'],
                    fold_data['val']['y'],   
                    fold_data['test']['X_robust'],
                    fold_data['test']['y'],  
                    fold_data['test']['dates'].values,
                    test_prices,
                    test_atr,
                    p_mult,
                    s_mult,
                    ml_models=ML_MODELS_CLASSIFICATION,
                    dl_models=DL_MODELS_CLASSIFICATION
                )
                
                summary = save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data)

                for model_name in ['CatBoost', 'LightGBM', 'XGBoost']:
                    ref = summary[summary['Model'] == model_name]
                    if ref.empty or not {'Test_TotalReturn_R', 'Test_Trades'}.issubset(ref.columns):
                        model_scores[model_name].append(-1e6)
                        continue

                    total_r = float(ref['Test_TotalReturn_R'].iloc[0])
                    trades  = float(ref['Test_Trades'].iloc[0])

                    if trades < 20:
                        score = -1e6
                    else:
                        score = total_r

                    model_scores[model_name].append(score)
                
                del evaluator, trainer
                gc.collect()
            
            model_mean_scores = {}
            for model_name, scores in model_scores.items():
                if len(scores) == 0:
                    model_mean_scores[model_name] = -1e6
                else:
                    model_mean_scores[model_name] = float(np.mean(scores))

            best_model = max(model_mean_scores, key=model_mean_scores.get)
            final_score = model_mean_scores[best_model]
            
            print(f"\n === Trial Finished. BestModel={best_model}, Final Score: {final_score:.4f}R ===")
            
            with open(LOG_PATH, "a") as f:
                f.write(f"{trial.number},{lookahead},{p_mult},{s_mult},{final_score}\n")
            
            new_row = pd.DataFrame([[trial.number, lookahead, p_mult, s_mult, final_score]], 
                                   columns=['trial','lookahead','profit_mult','stop_mult','score'])
            
            if existing_history.empty:
                existing_history = new_row
            else:
                existing_history = pd.concat([existing_history, new_row], ignore_index=True)
                
            return final_score
            
        except Exception as e:
            print(f" [Error] Trial Failed: {e}")
            traceback.print_exc()
            return 0.0

    study = optuna.create_study(direction='maximize')
    print(f"\n[Optuna] Starting optimization with {n_trials} trials...")
    study.optimize(objective, n_trials=n_trials)
    
    print("\n[Optuna] Optimization Completed!")
    print("Best Params:", study.best_params)
    return study



In [10]:
if __name__ == "__main__":
    if tf.config.list_physical_devices('GPU'):
        try:
            for gpu in tf.config.list_physical_devices('GPU'):
                tf.config.experimental.set_memory_growth(gpu, True)
            print(" GPU Detected & Memory Growth Set!")
        except RuntimeError as e:
            print(e)
    df_merged=pd.read_csv("merge_data.csv")
    df_hour=pd.read_csv("eth_hour.csv")
    if 'df_merged' in locals() and not df_merged.empty:
        df_merged['date'] = pd.to_datetime(df_merged['date'])
        df_merged = df_merged.sort_values('date').reset_index(drop=True)

        print(f"Loaded Data: {len(df_merged)} rows ({df_merged['date'].min().date()} ~ {df_merged['date'].max().date()})")

        study = run_optuna_optimization(df_merged,df_hour, n_trials=60)
        
        print("\n==================================================")
        print(f" Best Trial Value (Avg Buy Precision): {study.best_value:.4f}")
        print(f" Best Parameters: {study.best_params}")
        print("==================================================")
    else:
        print("[Error] 'df_merged' variable is not defined or empty. Please load data first.")


[I 2025-12-01 14:15:22,409] A new study created in memory with name: no-name-c212dcc4-4a77-4be4-b7be-cea8d4fa1867


 GPU Detected & Memory Growth Set!
Loaded Data: 2359 rows (2019-06-15 ~ 2025-11-28)

[Resume] Loaded 10 existing trials from model_results/2025-12-01/optuna_log.csv

[Optuna] Starting optimization with 60 trials...

 Starting NEW trial_0_141522_L7_P1.7_S1.2

 Pipeline Started... (Train Start: 2020-01-01)


/tmp/ipykernel_1761448/3336083649.py:57: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ta['EMA_12'] = ta.ema(close, length=12)


일봉 기간: 2019-06-15 ~ 2025-11-28
1시간봉 기간 (KST 변환 후): 2020-01-01 00:00:00 ~ 2025-11-28 22:00:00
[경고] 1시간봉 시작일(2020-01-01 00:00:00)이 일봉 시작일(2019-06-15 00:00:00) 이후입니다. 초반 타겟 손실 발생.
유효 샘플: 2339/2352 (제거: 13)
Win: 629 | Lose: 1710 | Win Rate: 26.89%
(2339, 88)

Reverse Rolling Walk-Forward + Final Holdout
Total period: 2020-01-01 ~ 2025-11-21 (2152 days)
Rolling train: 800d | Val: 90d | Test: 90d | Gap: 7d
Final holdout test: 2025-01-01 ~ 2025-11-21

Fold 1 (walk_forward_rolling)
  Train:  800d  2020-04-01 ~ 2022-06-09
  Val:     90d  2022-06-17 ~ 2022-09-14
  Test:    90d  2022-09-22 ~ 2022-12-20

Fold 2 (walk_forward_rolling)
  Train:  800d  2020-07-07 ~ 2022-09-14
  Val:     90d  2022-09-22 ~ 2022-12-20
  Test:    90d  2022-12-28 ~ 2023-03-27

Fold 3 (walk_forward_rolling)
  Train:  800d  2020-10-12 ~ 2022-12-20
  Val:     90d  2022-12-28 ~ 2023-03-27
  Test:    90d  2023-04-04 ~ 2023-07-02

Fold 4 (walk_forward_rolling)
  Train:  800d  2021-01-17 ~ 2023-03-27
  Val:     90d  2023-04-04 ~


[Feature Selection] Top 20 Features Selected:
 -> SMA_100, BB_WIDTH, usdt_totalCirculating_ma30_ratio, usdt_totalBridgedToUSD_ma30_ratio, usdt_totalMintedUSD_ma30_ratio, ATR_14, MACDH_12_26_9, eth_btc_corr_30d, MFI_14, OBV, l2_optimism_tvl_ma30_ratio, SMA_200, price_rank_250d, sp500_SP500_ma30_ratio, dxy_DXY_ma30_ratio, makerdao_makerdao_eth_tvl_ma30_ratio, PRICE_VS_LOW_60d, eth_btc_corr_7d, usdt_totalUnreleased_ma30_ratio, fg_fear_greed

 Processing Fold 6 (walk_forward_rolling_reverse)
 Train Period: 2021-07-30 ~ 2023-10-07 (N=800)
 Val   Period: 2023-10-15 ~ 2024-01-12 (N=90)
 Test  Period: 2024-01-20 ~ 2024-04-18 (N=90)
[Class Balance] Train Set: {0: 0.755, 1: 0.245}
[Multicollinearity] threshold=0.9
 -> 보호 피처 수(존재하는 것만): 32
 -> 예시(core_keep): ['VOLUME_RATIO', 'eth_btc_spread', 'PRICE_VS_LOW_5d', 'sp500_SP500_ma30_ratio', 'MACD_12_26_9', 'eth_return', 'SMA_200', 'gold_GOLD_ma30_ratio', 'PRICE_VS_LOW_60d', 'curve_curve-dex_eth_tvl_ma30_ratio']
 -> 제거 대상 피처 수: 9
 -> 제거 피처 예시: ['SMA_

...

일봉 기간: 2019-06-15 ~ 2025-11-28
1시간봉 기간 (KST 변환 후): 2020-01-01 00:00:00 ~ 2025-11-28 22:00:00
[경고] 1시간봉 시작일(2020-01-01 00:00:00)이 일봉 시작일(2019-06-15 00:00:00) 이후입니다. 초반 타겟 손실 발생.
유효 샘플: 2340/2353 (제거: 13)
Win: 395 | Lose: 1945 | Win Rate: 16.88%
(2340, 88)

Reverse Rolling Walk-Forward + Final Holdout
Total period: 2020-01-01 ~ 2025-11-22 (2153 days)
Rolling train: 800d | Val: 90d | Test: 90d | Gap: 6d
Final holdout test: 2025-01-01 ~ 2025-11-22

Fold 1 (walk_forward_rolling)
  Train:  800d  2020-01-10 ~ 2022-03-19
  Val:     90d  2022-03-26 ~ 2022-06-23
  Test:    90d  2022-06-30 ~ 2022-09-27

Fold 2 (walk_forward_rolling)
  Train:  800d  2020-04-15 ~ 2022-06-23
  Val:     90d  2022-06-30 ~ 2022-09-27
  Test:    90d  2022-10-04 ~ 2023-01-01

Fold 3 (walk_forward_rolling)
  Train:  800d  2020-07-20 ~ 2022-09-27
  Val:     90d  2022-10-04 ~ 2023-01-01
  Test:    90d  2023-01-08 ~ 2023-04-07

Fold 4 (walk_forward_rolling)
  Train:  800d  2020-10-24 ~ 2023-01-01
  Val:     90d  2023-01-08 ~


[Feature Selection] Top 20 Features Selected:
 -> SMA_100, BB_WIDTH, usdt_totalCirculating_ma30_ratio, usdt_totalMintedUSD_ma30_ratio, l2_arbitrum_tvl_ma30_ratio, ATR_14, curve_curve-dex_eth_tvl_ma30_ratio, PRICE_VS_HIGH_20d, SMA_200, eth_btc_corr_30d, usdt_totalUnreleased_ma30_ratio, sp500_SP500_ma30_ratio, MACD_12_26_9, fg_fear_greed, lido_lido_eth_tvl_ma30_ratio, MACDH_12_26_9, OBV, vix_VIX_ma30_ratio, usdt_totalBridgedToUSD_ma30_ratio, l2_optimism_tvl_ma30_ratio

 Processing Fold 6 (walk_forward_rolling_reverse)
 Train Period: 2021-05-04 ~ 2023-07-12 (N=800)
 Val   Period: 2023-07-19 ~ 2023-10-16 (N=90)
 Test  Period: 2023-10-23 ~ 2024-01-20 (N=90)
[Class Balance] Train Set: {0: 0.88, 1: 0.12}
[Multicollinearity] threshold=0.9
 -> 보호 피처 수(존재하는 것만): 32
 -> 예시(core_keep): ['VOLUME_RATIO', 'eth_btc_spread', 'PRICE_VS_LOW_5d', 'sp500_SP500_ma30_ratio', 'MACD_12_26_9', 'eth_return', 'SMA_200', 'gold_GOLD_ma30_ratio', 'PRICE_VS_LOW_60d', 'curve_curve-dex_eth_tvl_ma30_ratio']
 -> 제거 대상 피


[Feature Selection] Top 20 Features Selected:
 -> SMA_100, BB_WIDTH, l2_zksync era_tvl_ma30_ratio, l2_base_tvl_ma30_ratio, curve_curve-dex_eth_tvl_ma30_ratio, ATR_14, price_rank_250d, usdt_totalUnreleased_ma30_ratio, PRICE_VS_HIGH_5d, SMA_200, eth_btc_corr_30d, MACD_12_26_9, PRICE_VS_HIGH_20d, usdt_totalBridgedToUSD_ma30_ratio, dxy_DXY_ma30_ratio, fg_fear_greed, OBV, MFI_14, PRICE_VS_LOW_20d, usdt_totalCirculating_pct_1d

 Processing Fold 12 (walk_forward_rolling_reverse)
 Train Period: 2022-12-01 ~ 2025-02-07 (N=800)
 Val   Period: 2025-02-14 ~ 2025-05-14 (N=90)
 Test  Period: 2025-05-21 ~ 2025-08-18 (N=90)
[Class Balance] Train Set: {0: 0.8416666666666667, 1: 0.15833333333333333}
[Multicollinearity] threshold=0.9
 -> 보호 피처 수(존재하는 것만): 32
 -> 예시(core_keep): ['VOLUME_RATIO', 'eth_btc_spread', 'PRICE_VS_LOW_5d', 'sp500_SP500_ma30_ratio', 'MACD_12_26_9', 'eth_return', 'SMA_200', 'gold_GOLD_ma30_ratio', 'PRICE_VS_LOW_60d', 'curve_curve-dex_eth_tvl_ma30_ratio']
 -> 제거 대상 피처 수: 11
 -> 제거 피

[W 2025-12-01 14:18:45,090] Trial 5 failed with parameters: {'learning_rate': 0.033572701173918376, 'depth': 9, 'l2_leaf_reg': 2.562257087921245, 'bagging_temperature': 0.31626379709224106} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1761448/499541079.py", line 154, in objective
    model.fit(X_train, y_train, eval_set=(X_val, y_val))
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/catboost/core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/catboost/core.py", line 2410, in _fit
    self._train(
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/catboost/

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# 1. CSV 로드
df = pd.read_csv("merge_data.csv")      # 일봉
h  = pd.read_csv("eth_hour.csv")        # 1시간봉

# 2. datetime 파싱
df["date"] = pd.to_datetime(df["date"])
h["datetime"] = pd.to_datetime(h["datetime"])

# 3. 1시간봉을 KST로 변환 (+9h)
h["datetime_kst"] = h["datetime"] + pd.Timedelta(hours=9)
h["date_kst"] = h["datetime_kst"].dt.date

# 4. KST 기준 09:00인 1시간봉만 필터
h_9 = h[h["datetime_kst"].dt.hour == 9].copy()

# 5. 일봉에서 날짜만 추출
df_daily = df[["date", "ETH_Open"]].copy()
df_daily["date_only"] = df_daily["date"].dt.date

# 6. 날짜 기준으로 머지 (일봉 vs 09:00 1시간봉)
merged = pd.merge(
    df_daily,
    h_9[["date_kst", "datetime_kst", "open"]],
    left_on="date_only",
    right_on="date_kst",
    how="inner"
)

print("머지된 일수:", len(merged))

# 7. 시가 차이 확인
merged["open_diff"] = merged["ETH_Open"] - merged["open"]
print(merged["open_diff"].describe())

# 8. 차이가 큰 날만 샘플로 보기
mismatch = merged[merged["open_diff"].abs() > 1e-6].head(10)
print("\n차이가 나는 예시 (있다면):")
print(mismatch[["date", "datetime_kst", "ETH_Open", "open", "open_diff"]])


In [ ]:
df.tail(10)